In [59]:
import pandas
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import torchmetrics
import numpy as np

In [60]:
model = torchvision.models.resnet152(weights=True).cuda()

c:\Users\leomo\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [61]:
normalize = transforms.Compose(
    [transforms.Resize(256),
     transforms.CenterCrop(224),
     transforms.ToTensor(),
     transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    ])

imagenet = torchvision.datasets.ImageNet(root='data', split='val', transform=normalize)

In [62]:
#validation test split
val_set, test_set = torch.utils.data.random_split(imagenet, [int(len(imagenet)*0.2), int(len(imagenet)*0.8)])
val_loader = torch.utils.data.DataLoader(val_set, batch_size=8, shuffle=False, num_workers=3)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=8, shuffle=False, num_workers=3)


In [63]:
def predict(model, loader):
    model.eval()
    predictions = torch.tensor([])
    targets = torch.tensor([])
    with torch.no_grad():
        for i, (x, y) in enumerate(loader):
            x = x.cuda()
            y_hat = model(x)
            predictions = torch.cat((predictions, y_hat.cpu()))
            targets = torch.cat((targets, y.cpu()))
    return predictions, targets

In [64]:
predictions, targets = predict(model, test_loader)

In [65]:
accScore = torchmetrics.functional.accuracy(predictions, targets, task='multiclass', num_classes=1000)
print(f'Accuracy: {accScore.detach()}')

Accuracy: 0.784375011920929


In [66]:
from temp_scale import _ECELoss
print('ECE in test set: %.3f' % _ECELoss()(predictions, targets))
print('NLL in test set: %.3f' % F.cross_entropy(predictions, targets.type(torch.LongTensor)))

ECE in test set: 0.049
NLL in test set: 0.875


In [67]:
from temp_scale import ModelWithTemperature

temp_calib_model = ModelWithTemperature(model)

In [68]:
temp_calib_model = temp_calib_model.set_temperature(val_loader)

Before temperature - NLL: 0.883, ECE: 0.053
Optimal temperature: 1.379
After temperature - NLL: 0.870, ECE: 0.039


In [69]:
ts_predictions, ts_targets = predict(temp_calib_model, test_loader)

In [70]:
accScore = torchmetrics.functional.accuracy(ts_predictions, ts_targets, task='multiclass', num_classes=1000)
print(f'Accuracy: {accScore.detach()}')

Accuracy: 0.784375011920929


In [71]:
from temp_scale import _ECELoss
print('ECE in test set: %.3f' % _ECELoss()(ts_predictions, ts_targets))
print('NLL in test set: %.3f' % F.cross_entropy(ts_predictions, ts_targets.type(torch.LongTensor)))

ECE in test set: 0.041
NLL in test set: 0.864


In [72]:
cal_predictions, cal_targets = predict(temp_calib_model, val_loader)

In [73]:
#softmax for predictions tensor
val_smx = nn.Softmax(dim=1)(predictions)
val_ts_smx = nn.Softmax(dim=1)(ts_predictions)
cal_smx = nn.Softmax(dim=1)(cal_predictions)
#val_ts_smx = nn.Softmax(dim=1)(ts_test_predictions)

In [77]:
import conformal_logits

In [78]:
from conformal_logits import conformal, evaluate_conformal, create_fake_logits
conformal(cal_smx, cal_targets, val_smx, targets, len(cal_targets))

NameError: name 'np' is not defined

In [ ]:
cal_smx.shape

torch.Size([40000, 1000])